# Full example - hardcoded optimizer
- Hardcoded optimizer creating each variable manually
- Hardcoded optimizer creating each constraints manually
- There are created configuration tables in excel tables that the optimizer read

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Gurobi-ML-tips-modeling


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [3]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

In [4]:
import warnings
warnings.filterwarnings('ignore')

### 1. Load configuration file optimizer and configuration instance to solve
The are principally two kind of files to config optimizer
- **configuration file to create optimizer**: there a files used to create the optimization network such as, list of sets, list of variables, upper bound and lower bound, etc. Pricipally in this files should be parameters that doesn't change too much across the time. For example, the list of variable, is a parameter that if change, the structure of the network change and the machine learning models needs to change too

- **configuration file with instance to solve**: there files that change its values every time that the optimizer solve the problem. It represents the files with the actual values of the features, and so, this values change every time that the optimizer is executed

#### 1.1. IndexTime file

In [44]:
#################### define set ####################

## paths and files names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_indextime = 'IndexTime.xlsx'
path_indextime = path_folder_config_optimization + file_indextime

# read file
indextime = pd.read_excel(path_indextime)

# set index
index_set_time = pd.Index(indextime['IndexTime'].values)
index_set_time

Index(['t0', 't1', 't2', 't3', 't4', 't5', 't6'], dtype='object')

### 1.2 Decision variables

In [54]:
#################### define decision variables ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_allfeatures = 'AllFeatures.xlsx'
path_allfeatures = path_folder_config_optimization + file_allfeatures

# read file
allfeatures = pd.read_excel(path_allfeatures)

# table
allfeatures

,id,tag,feature_name,description,clasification_name,clasification,lower,upper,rate_change
0,1,X1,X1,Variable de entrada al proceso A. Variable Pri...,Primary,P,0.0,1000.0,100.0
1,2,O1,O1,Variable de entrada al proceso A. No es una va...,Observed,O,NaN,NaN,NaN
2,3,O2,O2,Variable de entrada al proceso A. No es una va...,Observed,O,NaN,NaN,NaN
3,4,O3,O3,Variable de entrada al proceso A. No es una va...,Observed,O,NaN,NaN,NaN
4,5,Y1,Y1,Variable target del proceso A y Variable de en...,Target,T,0.0,150.0,100.0
5,6,Z1,Z1,Variable de salida del tanque X y Variable de ...,Secundary,S,0.0,1000.0,100.0
6,7,X2,X2,Variable de entrada al proceso B. Aparece por ...,Primary,P,0.0,1000.0,100.0
7,8,O4,O4,Variable de entrada al proceso tanque TANK1. N...,Observed,O,NaN,NaN,NaN
8,9,O5,O5,Variable de entrada al proceso B. No es una va...,Observed,O,NaN,NaN,NaN
9,10,O6,O6,Variable de entrada al proceso B. No es una va...,Observed,O,NaN,NaN,NaN


In [46]:
##### define table to create decision vars
list_decision_var_names = ["X1", "X2", "Y1", "Z1", "tank_level"]
list_deceision_var_description = ["decision variable X1", "decision variable X2", "decision variable Y1", "decision variable Y2", "decision variable level tank"]
list_lower_bound = [0, 0, 0, 0, 200]
list_upper_bound = [1000, 1000, 1000, 1000, 1000]
list_rate_change = [100, np.nan, 40, 40, np.nan]

# dataframe
config_decision_var = pd.DataFrame()
config_decision_var['feature_name'] = list_decision_var_names
config_decision_var['description'] = list_deceision_var_description
config_decision_var['lower'] = list_lower_bound
config_decision_var['upper'] = list_upper_bound
config_decision_var['rate_change'] = list_rate_change

config_decision_var

,names,description,lower,upper,rate_change
0,X1,decision variable X1,0,1000,100.0
1,X2,decision variable X2,0,1000,NaN
2,Y1,decision variable Y1,0,1000,40.0
3,Z1,decision variable Y2,0,1000,40.0
4,tank_level,decision variable level tank,200,1000,NaN


In [47]:
stop

NameError: name 'stop' is not defined

#### 1.2. Configuration files instance to solve

In [48]:
# define file with initial values
list_decision_var_names = ["X1", "X2", "Y1", "Z1", "tank_level"]
list_initial_values = [10, 55, np.NaN, 20, 500]

# dataframe
config_initial_values = pd.DataFrame()
config_initial_values['feature_name'] = list_decision_var_names
config_initial_values['values'] = list_initial_values

config_initial_values

,names,values
0,X1,10.0
1,X2,55.0
2,Y1,NaN
3,Z1,20.0
4,tank_level,500.0


### 2. Create gurobi model

In [ ]:
# create model
model_opt = gp.Model('Example Optimization Model')

### 3. Create decision variables
It is necesary to have:
- list of elements of the sets of decision var
- table with the list of decision variables to create with important considerations when the are created (for example, upper bound and lower bound)
- **create the decision var and save it into a dictionary of decision var. Use the dictionary to call the decision var when they are used to create a constraints**

#### Aditional, when the decision var is created, set the initial value

- Fix the values of period t=0 for each decision var.

- t=0 represent the actual period or initial period and it in some problems and modelations is kwown

- In addition in this notebook, the values in time t=0 are fixed for all decision variables, inclusive if the decision var have a constraint that define its values in time t = 0 (so, this kind of constraints needs to be defined since t = 1)

In [ ]:
##### define a for across the configuration table to create the decision vars and save it into a python dictionary

decision_var = {}
for index_var in range(len(config_decision_var)):

    # get config values
    config_names_decision_var = config_decision_var.loc[index_var, 'feature_name']
    config_description_decision_var = config_decision_var.loc[index_var, 'description']
    print('defining decision variables: ', config_names_decision_var)

    # create decision var and save in the dictionary
    decision_var[config_names_decision_var] = gppd.add_vars(model_opt, 
                                                            index_set_time, 
                                                            name = config_description_decision_var,
                                                            #lb = -gp.GRB.INFINITY,
                                                            ub = gp.GRB.INFINITY
                                                           )

In [ ]:
decision_var

### 4. Set initial values decision variables
Set the initial values t=0 for the decision variables that needs this initial values and save in t=0

In [ ]:
# initial values decision variables - filter configuration file with only the decision var that have defined its initial values
config_initial_values_decision_var = config_initial_values[config_initial_values['values'].isnull() == False]
config_initial_values_decision_var = config_initial_values_decision_var.reset_index().drop(columns = 'index')
config_initial_values_decision_var

In [ ]:
for index_var in range(len(config_initial_values_decision_var)):

    # get config values
    config_names_decision_var = config_initial_values_decision_var.loc[index_var, 'feature_name']
    print('set initial values decision variables: ', config_names_decision_var)

    # get initial value
    initial_value_decision_var = config_initial_values_decision_var[config_initial_values_decision_var['feature_name'] == config_names_decision_var]['values'].values[0]
    
    # set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined
    model_opt.addConstr(decision_var[config_names_decision_var]['t0'] == initial_value_decision_var,  
                        name = f'Initial Value {config_names_decision_var}')

### 5. Lower bound and Upper bound decision variables
For example, lower bound and upper bound of tank level

---
If one decision variables doesn't need a lower bound and upper bound parameter, its value in configuration file is saved as np.NaN

Then filter the configuration table to have only the decision variables that have defined a rate_change

define in differents files one for lower bounds and another for upper bounds

#### 5.1 Lower bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [ ]:
# lower bounds parameters - filter configuration file with only the decision var that have defined its lower bounds
config_decision_var_lower_bounds = config_decision_var[config_decision_var['lower'].isnull() == False]
config_decision_var_lower_bounds = config_decision_var_lower_bounds.reset_index().drop(columns = 'index')
config_decision_var_lower_bounds

In [ ]:
# generate constaint - lower bound

for index_var in range(len(config_decision_var_lower_bounds)):

    # get config values
    config_names_decision_var = config_decision_var_lower_bounds.loc[index_var, 'feature_name']
    print('lower bound decision_var: ', config_names_decision_var)
    
    gppd.add_constrs(model_opt, 
                     decision_var[config_names_decision_var],  # decision var
                     gp.GRB.GREATER_EQUAL,
                     config_decision_var_lower_bounds[config_decision_var_lower_bounds['feature_name'] == config_names_decision_var]['lower'].values[0],  # lower bound
                     name = f'Lower bound {config_names_decision_var}')

#### 5.2 upper bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [ ]:
# upper bounds parameters - filter configuration file with only the decision var that have defined its upper bounds
config_decision_var_upper_bounds = config_decision_var[config_decision_var['upper'].isnull() == False]
config_decision_var_upper_bounds = config_decision_var_upper_bounds.reset_index().drop(columns = 'index')
config_decision_var_upper_bounds

In [ ]:
# generate constaint - upper bound

for index_var in range(len(config_decision_var_upper_bounds)):

    # get config values
    config_names_decision_var = config_decision_var_upper_bounds.loc[index_var, 'feature_name']
    print('upper bound decision_var: ', config_names_decision_var)
    
    gppd.add_constrs(model_opt, 
                     decision_var[config_names_decision_var],  # decision var
                     gp.GRB.LESS_EQUAL, 
                     config_decision_var_upper_bounds[config_decision_var_upper_bounds['feature_name'] == config_names_decision_var]['upper'].values[0],  # upper bound
                     name = f'Upper bound {config_names_decision_var}')

### 6. Rate change of decision variables across the time

If one decision variables doesn't need a rate change parameter, its value in configuration file is saved as np.NaN

Then filter the configuration table to have only the decision variables that have defined a rate_change

\begin{align}
&| ~ X^{t}_{i} - X^{t-1}_{i} ~ | ~ \leq  ~ c_{i} &\quad \forall ~ i \in I, t \in T \tag{6}\\
\end{align}

#### 6.1 define table with values of rate_change

In [ ]:
# rate change parameters - filter configuration file with only the decision var that have defined its rates changes
config_decision_var_rate_change = config_decision_var[config_decision_var['rate_change'].isnull() == False]
config_decision_var_rate_change = config_decision_var_rate_change.reset_index().drop(columns = 'index')
config_decision_var_rate_change

#### 6.2 The rate change constraints is defined using absolute values. So it is necesary create an auxiliar decision variable


This auxiliar decision var created represents the difference between the decisions vars. It is necesary create auxiliar variables and fixed it first value into zero (set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined)

In this example
- diff_time_x >= (x(t-1) - x(t))

- diff_time_x >= -(x(t-1) - x(t))

- diff_time_x <= delta_x

In [ ]:
### create an auxiliar decion var "diff" for each decision var that has defined its rate change

aux_decision_var = {}
for index_var in range(len(config_decision_var_rate_change)):

    # get config values
    config_names_decision_var = config_decision_var_rate_change.loc[index_var, 'feature_name']
    print('create auxiliar variable diff "t" - "t-1": ', config_names_decision_var)

    # create decision var and save in the dictionary
    aux_decision_var[config_names_decision_var] = gppd.add_vars(model_opt, 
                                                                index_set_time, 
                                                                name = f'diff "t" - "t_1" of decision var: {config_names_decision_var}',
                                                                lb = -gp.GRB.INFINITY,
                                                                ub = gp.GRB.INFINITY
                                                               )

    # set initial value (diff t = 0 and t = -1) is set to cero because t = -1 is not defined
    model_opt.addConstr(aux_decision_var[config_names_decision_var]['t0'] == 0,  name = f'Initial Value diff {config_names_decision_var}')

#### 6.3 Define rate change variable constraint for each decision variable and for each time

In [ ]:
# for each variable
for index_var in range(len(config_decision_var_rate_change)):

    # get config values
    config_names_decision_var = config_decision_var_rate_change.loc[index_var, 'feature_name']
    print('rate change decision var: ', config_names_decision_var)

    # get rate change for this decision var
    rate_change_decision_var = config_decision_var_rate_change[config_decision_var_rate_change['feature_name'] == config_names_decision_var]['rate_change'].values[0]
    
    # for each time in this decision variable
    for index_time in range(1, len(index_set_time)):
        
        ### define time t and t-1
        time_t = index_set_time[index_time]
        time_t_1 = index_set_time[index_time-1]
    
        ### define constraints
        # positive segment
        model_opt.addConstr(aux_decision_var[config_names_decision_var][time_t] >= (decision_var[config_names_decision_var][time_t] - decision_var[config_names_decision_var][time_t_1]), 
                            name = f'diff {config_names_decision_var} positive segment {time_t} - {time_t_1}')

        # negative segment
        model_opt.addConstr(aux_decision_var[config_names_decision_var][time_t] >= -(decision_var[config_names_decision_var][time_t] - decision_var[config_names_decision_var][time_t_1]), 
                            name = f'diff {config_names_decision_var} negative segment {time_t} - {time_t_1}')

        # rate change
        model_opt.addConstr(aux_decision_var[config_names_decision_var][time_t] <= rate_change_decision_var, 
                            name = f'diff_var_X1 delta {time_t} - {time_t_1}')

### 7. Volumen change (Inventory change) - across time - relation between decision variables

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ \leq  ~ UB_{j} &\quad \forall ~ t \in T, j \in J \tag{2}\\
\end{align}

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ \geq  ~ LB_{j} &\quad \forall ~ t \in T, j \in J \tag{3}\\
\end{align}

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ =  ~ V^{t}_{j} &\quad \forall ~ t \in T, j \in J \tag{4}\\
\end{align}

## 7.0 IMPORTANT: The two first contraints can be replaced with the constraint that defined the lower and upper bound of the decision variables.
So the only constraint that needs to be defined is:
## THIS PART IS DIFFERENT AGAINS THE BASE CODES

\begin{align}
&V^{t-1}_{j} ~ + \sum_{i}  Y^{t}_{i,j} -  \sum_{i}  Z^{t}_{i',j} ~ =  ~ V^{t}_{j} &\quad \forall ~ t \in T, j \in J \tag{4}\\
\end{align}

#### 7.1 Define full constraints - automatically all times
- Automatization constraint. For across all the set time (t0, t1, t2, etc). But it is important to see that the first value of the decision variable (period t=0) is defined before, so the "for cicle" start from (t1, t2, etc)


- **IMPORTANT**: This codes are defined for only this example when there are only one input and one output. In the future it needs to genelize to recibe multiple input and output values

- **IMPORTANT 2**: There are only one tank in the example, so it is more easy to define the automatization

- **IMPORTANT 3**: in this example the map of the input and output variables of the tank is HARCODED. In the full example it needs to be parametrize

original codes
    
    model_opt.addConstr(var_tank_level[time_t_1] + var_Y1[time_t] - var_Z1[time_t] == var_tank_level[time_t], 
                name = f'new level of tank {time_t} - {time_t_1}')

In [ ]:
for index_time in range(1, len(index_set_time)):
    
    ### define time t and t-1
    time_t = index_set_time[index_time]
    time_t_1 = index_set_time[index_time-1]

    ### define constraints
    model_opt.addConstr(decision_var['tank_level'][time_t_1] + decision_var['Y1'][time_t] - decision_var['Z1'][time_t] == decision_var['tank_level'][time_t], 
                name = f'new level of tank {time_t} - {time_t_1}')

### 8. Load a Machine Learning Model as constraints that represent the relations in the process
Add a constraint defined as a machine learning model that represent the relation between X1, X2 with the output of the process (Y1)

#### 8.1 Load machine learning model
In this example doesn't exist a ML model. So a dummy model is created,.

In this example, the model is trained with X1, X2, 01. Where X1, X2 are decision variables and O1 are observed variables

In [ ]:
# generate example data
X_train = pd.DataFrame([
    [100, 1, 2],
    [50, 2, 30],
    [80, 1, 25],
    [70, 3, 23]
], columns = ['decision variable X1', 'decision variable X2', 'O1'])

y_train = pd.DataFrame([
    [40],
    [20],
    [33],
    [30]
], columns = ['target'])

# train lr model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
r2 = lr.score(X_train, y_train)
print('r2 score:', r2)

# get model
model_ml = lr

#### 8.2 Generate instance of machine learning model

In [ ]:
# generate instance observed variables - no decision variables of optimization
instance_observed_value = 15
instance_observed = pd.DataFrame(instance_observed_value, columns = ['O1'], index = index_set_time)

# generate instance controlable variables - decision variables of optimization
instance_controlables = pd.DataFrame([decision_var['X1'], decision_var['X2']]).T

# append into one dataframe
instance = pd.concat([instance_observed, instance_controlables], axis = 1)

# define a list with the order
list_order_features_ml_model = ['decision variable X1', 'decision variable X2', 'O1']

# sort instance according order features
instance = instance[list_order_features_ml_model]

instance

#### 8.3 Add machine learning model as constraint
OBS 1. As you can see, this step is not parametrized neither automatized

OBS 2. This is the unique constraint that save the constraint as a variable (python variable or python dictionary as the decision variables are saved).

In [ ]:
pred_constr = add_predictor_constr(gp_model = model_opt, 
                     predictor = model_ml, 
                     input_vars = instance, # instance pandas gurobi
                     output_vars = decision_var['Y1'], # target
                     name = f'model_predict Y1'
                    )

pred_constr.print_stats()

### 8. Define a custom function as constraints that represent the relations in the process
Add a constraint defined as a funtion that represent the relation between X1, X2 with the output of the process (Y1)

**Important: this constraint and the objective functions are more hardoded that the rest of constraints**

In [ ]:
# define parameters of the constraint
alpha_feature_x1 = 1/5
alpha_feature_x2 = 15

In [ ]:
# define function as constraint
gppd.add_constrs(model_opt, 
                 (alpha_feature_x1 * decision_var['X1'] + alpha_feature_x1 * decision_var['X1']), 
                 gp.GRB.EQUAL, 
                 decision_var['Y1'], 
                 name = 'function as constraint output process'
                )

In [ ]:
model_opt

In [ ]:
model_opt.update()

In [ ]:
model_opt

### 9. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
# original
# model_opt.setObjective(var_tank_level.sum(),
#                        gp.GRB.MINIMIZE)

model_opt.setObjective(decision_var['tank_level'].sum(),
                       gp.GRB.MINIMIZE)

## ----> return the mdoel_opt that has defined for this instance can get the solution <----

In [ ]:
model_opt

### 10. Optimize and get optimal values

In [ ]:
# solve
model_opt.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
model_opt.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################
solution["var_X1"] = decision_var['X1'].gppd.X
solution["var_X2"] = decision_var['X2'].gppd.X
solution["var_Y1"] = decision_var['Y1'].gppd.X
solution["var_Z1"] = decision_var['Z1'].gppd.X
solution["var_tank_level"] = decision_var['tank_level'].gppd.X


######################## # get value objetive function ########################
opt_objetive_function = model_opt.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution